In [1]:
import cv2
import skimage.io as io
import numpy as np
from skimage.morphology import binary_erosion, binary_dilation, binary_closing,skeletonize, thin
from scipy.signal import convolve2d
from statistics import mode 
from math import sqrt

In [2]:
def showBinaryImg(img):
    cv2.imshow("Images",img*255)
    cv2.waitKey(0) 
    cv2.destroyAllWindows()
def showImg(img):
    cv2.imshow("Images",img)
    cv2.waitKey(0) 
    cv2.destroyAllWindows()

In [3]:
def calcStaffHeightSpace(img):
    count=0
    staffHeight=[]
    staffSpace=[]
    n,m=img.shape
    lines=[]
    for j in range(m):
        start=end=0
        for i in range(n-1):
            if(img[i][j]!=img[i+1][j]):
                if img[i][j]==0:
                    #end of zeros
                    start=i
                    staffSpace.append(count)
                else:
                    #end of ones
                    end=i
                    lines.append([j,start,end,count])#colIndex,start in coloum Index,end in coloum index,thikness
                    staffHeight.append(count)
                count=0
            else:
                count+=1
        
                       
    staffHeight=mode(staffHeight)
    staffSpace=mode(staffSpace)
    return lines,staffSpace,staffHeight


In [4]:
def removeSymbol(img,lines,staffHeight,staffSpace):
    Thres=min(staffHeight+staffSpace,2*staffHeight)
    lines=np.array(lines)
    indices=np.where(lines[:,3]>Thres)
    staffImg=np.copy(img)
    for i in indices[0]:
       # print(i)
        col,start,end,thres=lines[i]
        staffImg[start:end+1,col]=0
    #print(img)
    return staffImg

In [5]:
img= cv2.imread("Binarized.png",0) 
img=img/255
# print(img)
img=1-img
imgLines,staffS,staffH=calcStaffHeightSpace(img)
# print(staffH,staffS)
imgRemovedSymbols=removeSymbol(img,imgLines,staffH,staffS)
cv2.imwrite("staffout.png",imgRemovedSymbols*255)
#showBinaryImg(imgRemovedSymbols)
neg=img-imgRemovedSymbols
print(img)
print(imgRemovedSymbols)
cv2.imwrite("neg.png",neg*255)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


True

In [7]:
def getDist(curComp,compIndex,lines,j,staffHremoved):
    indices=np.where((lines[:,0]==j) & (lines[:,3] >= 0.5*staffHremoved) & (lines[:,3] <= 2*staffHremoved))
    if(compIndex>=indices[0].size):
        return 0
    nextComp=lines[indices[0][compIndex]]
    colIndex1=curComp[0]
    colIndex2=nextComp[0]
    rowIndex1=(curComp[2]+curComp[1])/2
    rowIndex2=(nextComp[2]+nextComp[1])/2
    dist=sqrt((colIndex1-colIndex2)**2+(rowIndex1-rowIndex2)**2)
    return (1/j)*dist

imgLines,staffSremoved,staffHremoved=calcStaffHeightSpace(imgRemovedSymbols)
lines=np.array(imgLines)
indices=np.where(lines[:,0]==5)
k=5
oGlobal=[]
m=imgRemovedSymbols.shape[1]
for i in range(m):
    o=[]
    indices=np.where((lines[:,0]==i) & (lines[:,3] >= 0.5*staffHremoved) & (lines[:,3] <= 2*staffHremoved))
    curCompIndex=0
    if indices[0].size > 5:
        for component in indices[0]:
            col=lines[component][0]
            orient=0
            for j in range(k):
                if(col+j+1<m):

                    orient+=getDist(lines[component],curCompIndex,lines,col+j+1,staffHremoved)
            o.append(orient)
            curCompIndex+=1
        oi=sum(o)/indices[0].size
        oGlobal.append(oi)
print(oGlobal)   
n=img.shape[0]
groups=np.zeros((n,m))
lastGroup=0
def getNextPixel(slope,x,y):
    c=y-slope*x
    neigh0=[x,y+1]
    neigh1=[x-1,y+1]
    neigh7=[x+1,y+1]
    dist0=abs(neigh0[1]-slope*neigh0[0]-c)/sqrt(1+slope**2)
    dist1=abs(neigh1[1]-slope*neigh1[0]-c)/sqrt(1+slope**2)
    dist7=abs(neigh7[1]-slope*neigh7[0]-c)/sqrt(1+slope**2)

    if(dist0<dist1 and dist0<dist7):
        return x,y+1
    if(dist1<dist0 and dist1<dist7):
        return x-1,y+1
    if(dist7<dist0 and dist7<dist1):
        return x+1,y+1
    return x+1,y+1
#for x in range(n):
#    for y in range(m):
#        if(groups[x][y]==0):
#            groups[x][y]=lastGroup+1
#            lastGroup+=1
#            for orien in range(len(oGlobal)):
#                xn,yn=getNextPixel(oGlobal[orien],x,y)
#                if(xn>=n or yn >=m):
#                    break
#                groups[xn][yn]=groups[x][y]
        


[0.41637421242552525, 0.5051700521486029, 0.743416251026229, 0.5483996344600718, 0.6095301258146585, 0.6462624161355628, 0.7689976088114955, 0.56629519918182, 1.1623394161609413, 1.2823409821174327, 0.6650304858115741, 0.7792845865177223, 0.4077411323400856, 0.5521840159953166, 0.5784637835165237, 1.0186177079621226, 0.6685193406389685, 0.659140974810149, 0.8651208932098112, 0.589128166667688, 1.4372578510178433, 0.7216889527919386, 0.7585962230603531, 0.8075118235716735, 0.70425333122697, 1.1656553736913762, 0.6978742387123987, 0.8301976110044251, 0.7097155615974748, 0.9476498513469898, 1.2771156458554338, 0.6379325314049571, 0.364048486094882, 0.5460308054756139, 0.4024325622646117, 0.5199943866362817, 0.4119410710024759, 0.8142123886960986, 0.4868752621537538, 0.701922308662457, 0.7940950456095681, 0.4518466131087756, 0.48081589097426475, 0.6073299103324529, 0.5012402736736662, 0.5651138577846023, 0.52592087579624, 0.7264764092830776, 0.9913520731007157, 0.41475451157275345, 0.24437